In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4>Start cells here if {state}_geo file has not been FIPS encoded

In [2]:
fips_check = pd.read_csv("../../data/state_data/geo/geocoded/geo_AS.csv") #- original geocoded file
fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
fips_check

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,19445,6927458108,7/22/20,951,PPP,AST TELECOM LLC,478 Laufou Shopping Centre,PAGO PAGO,AS,96799.0,...,999 Other,5179 Other Telecommunications,5179.0,HAWAII DISTRICT OFFICE,"b $350,000-1 million",100 to 249,"478 Laufou Shopping Centre, PAGO PAGO, AS",-14.312881,-170.714627,NaN
1,19446,7124948005,6/30/20,951,PPP,MC CONNELL DOWELL,4669 TAFUNA,PAGO PAGO,AS,96799.0,...,999 Other,2362 Nonresidential Building Construction,2362.0,HAWAII DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"4669 TAFUNA, PAGO PAGO, AS",-14.331296,-170.726985,NaN
2,19447,1760228005,6/23/20,951,PPP,PARAMOUNT BUILDERS LIMITED,POB 2070,PAGO PAGO,AS,96799.0,...,999 Other,2362 Nonresidential Building Construction,2362.0,HAWAII DISTRICT OFFICE,"b $350,000-1 million",100 to 249,"POB 2070, PAGO PAGO, AS",-14.275632,-170.702036,NaN
3,19448,2530867306,4/29/20,951,PPP,NANA'S INC,POB 5350,PAGO PAGO,AS,96799.0,...,999 Other,2361 Residential Building Construction,2361.0,HAWAII DISTRICT OFFICE,"b $350,000-1 million",100 to 249,"POB 5350, PAGO PAGO, AS",-14.275632,-170.702036,NaN
4,19449,2121297104,4/10/20,951,PPP,AMERICAN SAMOA 2000 INC,PO BOX 4047,PAGO PAGO,AS,96799.0,...,NaN,NaN,NaN,HAWAII DISTRICT OFFICE,"b $350,000-1 million",100 to 249,"PO BOX 4047, PAGO PAGO, AS",-14.275632,-170.702036,NaN
5,19450,4621778206,8/6/20,951,PPP,"PETER E REID STEVEDORING, INC.",1030 FAGATOGO,PAGO PAGO,AS,96799.0,...,999 Other,4841 General Freight Trucking,4841.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",100 to 249,"1030 FAGATOGO, PAGO PAGO, AS",-14.279568,-170.692331,NaN
6,19451,4195837200,4/27/20,951,PPP,FORSGREN LTD. INC,POB 1358,PAGO PAGO,AS,96799.0,...,NaN,NaN,NaN,HAWAII DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"POB 1358, PAGO PAGO, AS",-14.275632,-170.702036,NaN
7,19452,4181737204,4/27/20,951,PPP,OTTOVILLE INVESTMENTS ONE INC,POB 999 OTTOVILLE ROAD,PAGOPAGO,AS,96799.0,...,999 Other,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"POB 999 OTTOVILLE ROAD, PAGOPAGO, AS",-14.275632,-170.702036,NaN
8,19453,1741728203,7/30/20,951,PPP,CONGREGATION CHRISTIAN CHURCH IN AMERICAN SAMOA,POB 1537,PAGO PAGO,AS,96799.0,...,999 Other,8131 Religious Organizations,8131.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"POB 1537, PAGO PAGO, AS",-14.275632,-170.702036,NaN
9,19454,2520817309,4/29/20,951,PPP,HARBOR MARITIME & STEVEDORING CO,POB 5350,PAGO PAGO,AS,96799.0,...,999 Other,4883 Support Activities for Water Transportation,4883.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",100 to 249,"POB 5350, PAGO PAGO, AS",-14.275632,-170.702036,NaN


In [3]:
columns = ['LoanNumber','full_add', 'Lat', 'Long']
fips_check = fips_check[columns]

In [4]:
fips_check_nan = fips_check[fips_check["Lat"].isnull()] # check to make sure no nulls in coords
fips_check_nan #3

,LoanNumber,full_add,Lat,Long


In [5]:
fips_check_notna = fips_check[fips_check["Lat"].notna()] # check to make sure no nulls in coords
fips_check_notna.head() #15866

,LoanNumber,full_add,Lat,Long
0,6927458108,"478 Laufou Shopping Centre, PAGO PAGO, AS",-14.312881,-170.714627
1,7124948005,"4669 TAFUNA, PAGO PAGO, AS",-14.331296,-170.726985
2,1760228005,"POB 2070, PAGO PAGO, AS",-14.275632,-170.702036
3,2530867306,"POB 5350, PAGO PAGO, AS",-14.275632,-170.702036
4,2121297104,"PO BOX 4047, PAGO PAGO, AS",-14.275632,-170.702036


<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [ ]:
fips_check_scraped = pd.read_csv("../../data/state_data/geo/geo_fips/AS_fips_scraped.csv") 
fips_check_scraped.head() #15869

In [ ]:
columns = ['LoanNumber','full_add', 'Lat', 'Long', 'FIPS_z']
fips_check_scraped = fips_check_scraped[columns]

In [28]:
#pd.set_option('display.float_format', '{:.0f}'.format)

In [ ]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna #15866

In [ ]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #5503

<h4>FIPS Script

In [6]:
#if geocode file
lat = fips_check_notna["Lat"].astype(float).astype(str).to_list()
long = fips_check_notna["Long"].astype(float).astype(str).to_list()

#if FIPS_z file 
#lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
#long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [7]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
#driver = webdriver.Chrome(service=BraveService(ChromeDriverManager(chrome_type=ChromeType.BRAVE).install()))
#executable_path = {'executable_path': webdriver.Chrome(service=BraveService(ChromeDriverManager(chrome_type=ChromeType.BRAVE).install()))}
#executable_path = {'executable_path': ChromeDriverManager(chrome_type=ChromeType.BRAVE).install()}
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('60')] #AS
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

The version of chrome cannot be detected. Trying with latest driver version


  0%|          | 0/20 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [8]:
geo_id_list =geo_id 

Length Check

In [9]:
print(len(geo_id_list))

20


Matching FIPS to DataFrame

In [10]:
#for geocode file
fips_fill = fips_check_notna #confirm pair to notna DataFrame
#if FIPS available
#fips_fill = fips_check_scraped_null.iloc[:25062]

In [11]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

,LoanNumber,full_add,Lat,Long,FIPS_z
0,6927458108,"478 Laufou Shopping Centre, PAGO PAGO, AS",-14.312881,-170.714627,[600109509003002]
1,7124948005,"4669 TAFUNA, PAGO PAGO, AS",-14.331296,-170.726985,[600509512021009]
2,1760228005,"POB 2070, PAGO PAGO, AS",-14.275632,-170.702036,[600109506002001]
3,2530867306,"POB 5350, PAGO PAGO, AS",-14.275632,-170.702036,[600109506002001]
4,2121297104,"PO BOX 4047, PAGO PAGO, AS",-14.275632,-170.702036,[600109506002001]
5,4621778206,"1030 FAGATOGO, PAGO PAGO, AS",-14.279568,-170.692331,[600109507002001]
6,4195837200,"POB 1358, PAGO PAGO, AS",-14.275632,-170.702036,[600109506002001]
7,4181737204,"POB 999 OTTOVILLE ROAD, PAGOPAGO, AS",-14.275632,-170.702036,[600109506002001]
8,1741728203,"POB 1537, PAGO PAGO, AS",-14.275632,-170.702036,[600109506002001]
9,2520817309,"POB 5350, PAGO PAGO, AS",-14.275632,-170.702036,[600109506002001]


Remove brackets

In [12]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

,LoanNumber,full_add,Lat,Long,FIPS_z
0,6927458108,"478 Laufou Shopping Centre, PAGO PAGO, AS",-14.312881,-170.714627,600109509003002
1,7124948005,"4669 TAFUNA, PAGO PAGO, AS",-14.331296,-170.726985,600509512021009
2,1760228005,"POB 2070, PAGO PAGO, AS",-14.275632,-170.702036,600109506002001
3,2530867306,"POB 5350, PAGO PAGO, AS",-14.275632,-170.702036,600109506002001
4,2121297104,"PO BOX 4047, PAGO PAGO, AS",-14.275632,-170.702036,600109506002001
5,4621778206,"1030 FAGATOGO, PAGO PAGO, AS",-14.279568,-170.692331,600109507002001
6,4195837200,"POB 1358, PAGO PAGO, AS",-14.275632,-170.702036,600109506002001
7,4181737204,"POB 999 OTTOVILLE ROAD, PAGOPAGO, AS",-14.275632,-170.702036,600109506002001
8,1741728203,"POB 1537, PAGO PAGO, AS",-14.275632,-170.702036,600109506002001
9,2520817309,"POB 5350, PAGO PAGO, AS",-14.275632,-170.702036,600109506002001


In [13]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #24

,LoanNumber,full_add,Lat,Long,FIPS_z


Parse for Merge

In [14]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
0,6927458108,600109509003002
1,7124948005,600509512021009
2,1760228005,600109506002001
3,2530867306,600109506002001
4,2121297104,600109506002001
5,4621778206,600109507002001
6,4195837200,600109506002001
7,4181737204,600109506002001
8,1741728203,600109506002001
9,2520817309,600109506002001


Read in base data

In [15]:
#if from geocode file
fips_check = pd.read_csv("../../data/state_data/geo/geocoded/geo_AS.csv")
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1"], axis =1) 
fips_check.head()

#if from FIPS file
#fips_check = pd.read_csv("../../data/state_data/geo/geo_fips/AS_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
#fips_check.head() #22826


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,19445,19445,19445,6927458108,7/22/20,951,PPP,AST TELECOM LLC,478 Laufou Shopping Centre,PAGO PAGO,...,999 Other,5179 Other Telecommunications,5179.0,HAWAII DISTRICT OFFICE,"b $350,000-1 million",100 to 249,"478 Laufou Shopping Centre, PAGO PAGO, AS",-14.312881,-170.714627,NaN
1,19446,19446,19446,7124948005,6/30/20,951,PPP,MC CONNELL DOWELL,4669 TAFUNA,PAGO PAGO,...,999 Other,2362 Nonresidential Building Construction,2362.0,HAWAII DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"4669 TAFUNA, PAGO PAGO, AS",-14.331296,-170.726985,NaN
2,19447,19447,19447,1760228005,6/23/20,951,PPP,PARAMOUNT BUILDERS LIMITED,POB 2070,PAGO PAGO,...,999 Other,2362 Nonresidential Building Construction,2362.0,HAWAII DISTRICT OFFICE,"b $350,000-1 million",100 to 249,"POB 2070, PAGO PAGO, AS",-14.275632,-170.702036,NaN
3,19448,19448,19448,2530867306,4/29/20,951,PPP,NANA'S INC,POB 5350,PAGO PAGO,...,999 Other,2361 Residential Building Construction,2361.0,HAWAII DISTRICT OFFICE,"b $350,000-1 million",100 to 249,"POB 5350, PAGO PAGO, AS",-14.275632,-170.702036,NaN
4,19449,19449,19449,2121297104,4/10/20,951,PPP,AMERICAN SAMOA 2000 INC,PO BOX 4047,PAGO PAGO,...,NaN,NaN,NaN,HAWAII DISTRICT OFFICE,"b $350,000-1 million",100 to 249,"PO BOX 4047, PAGO PAGO, AS",-14.275632,-170.702036,NaN


Merge with MainFrame

In [16]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,19445,19445,19445,6927458108,7/22/20,951,PPP,AST TELECOM LLC,478 Laufou Shopping Centre,PAGO PAGO,...,5179 Other Telecommunications,5179.0,HAWAII DISTRICT OFFICE,"b $350,000-1 million",100 to 249,"478 Laufou Shopping Centre, PAGO PAGO, AS",-14.312881,-170.714627,NaN,600109509003002
1,19446,19446,19446,7124948005,6/30/20,951,PPP,MC CONNELL DOWELL,4669 TAFUNA,PAGO PAGO,...,2362 Nonresidential Building Construction,2362.0,HAWAII DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"4669 TAFUNA, PAGO PAGO, AS",-14.331296,-170.726985,NaN,600509512021009
2,19447,19447,19447,1760228005,6/23/20,951,PPP,PARAMOUNT BUILDERS LIMITED,POB 2070,PAGO PAGO,...,2362 Nonresidential Building Construction,2362.0,HAWAII DISTRICT OFFICE,"b $350,000-1 million",100 to 249,"POB 2070, PAGO PAGO, AS",-14.275632,-170.702036,NaN,600109506002001
3,19448,19448,19448,2530867306,4/29/20,951,PPP,NANA'S INC,POB 5350,PAGO PAGO,...,2361 Residential Building Construction,2361.0,HAWAII DISTRICT OFFICE,"b $350,000-1 million",100 to 249,"POB 5350, PAGO PAGO, AS",-14.275632,-170.702036,NaN,600109506002001
4,19449,19449,19449,2121297104,4/10/20,951,PPP,AMERICAN SAMOA 2000 INC,PO BOX 4047,PAGO PAGO,...,NaN,NaN,HAWAII DISTRICT OFFICE,"b $350,000-1 million",100 to 249,"PO BOX 4047, PAGO PAGO, AS",-14.275632,-170.702036,NaN,600109506002001
5,19450,19450,19450,4621778206,8/6/20,951,PPP,"PETER E REID STEVEDORING, INC.",1030 FAGATOGO,PAGO PAGO,...,4841 General Freight Trucking,4841.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",100 to 249,"1030 FAGATOGO, PAGO PAGO, AS",-14.279568,-170.692331,NaN,600109507002001
6,19451,19451,19451,4195837200,4/27/20,951,PPP,FORSGREN LTD. INC,POB 1358,PAGO PAGO,...,NaN,NaN,HAWAII DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"POB 1358, PAGO PAGO, AS",-14.275632,-170.702036,NaN,600109506002001
7,19452,19452,19452,4181737204,4/27/20,951,PPP,OTTOVILLE INVESTMENTS ONE INC,POB 999 OTTOVILLE ROAD,PAGOPAGO,...,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"POB 999 OTTOVILLE ROAD, PAGOPAGO, AS",-14.275632,-170.702036,NaN,600109506002001
8,19453,19453,19453,1741728203,7/30/20,951,PPP,CONGREGATION CHRISTIAN CHURCH IN AMERICAN SAMOA,POB 1537,PAGO PAGO,...,8131 Religious Organizations,8131.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"POB 1537, PAGO PAGO, AS",-14.275632,-170.702036,NaN,600109506002001
9,19454,19454,19454,2520817309,4/29/20,951,PPP,HARBOR MARITIME & STEVEDORING CO,POB 5350,PAGO PAGO,...,4883 Support Activities for Water Transportation,4883.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",100 to 249,"POB 5350, PAGO PAGO, AS",-14.275632,-170.702036,NaN,600109506002001


<b>Fill FIPS Columns - if FIPS file

In [17]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,19445,19445,19445,6927458108,7/22/20,951,PPP,AST TELECOM LLC,478 Laufou Shopping Centre,PAGO PAGO,...,5179.0,HAWAII DISTRICT OFFICE,"b $350,000-1 million",100 to 249,"478 Laufou Shopping Centre, PAGO PAGO, AS",-14.312881,-170.714627,NaN,600109509003002,600109509003002
1,19446,19446,19446,7124948005,6/30/20,951,PPP,MC CONNELL DOWELL,4669 TAFUNA,PAGO PAGO,...,2362.0,HAWAII DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"4669 TAFUNA, PAGO PAGO, AS",-14.331296,-170.726985,NaN,600509512021009,600509512021009
2,19447,19447,19447,1760228005,6/23/20,951,PPP,PARAMOUNT BUILDERS LIMITED,POB 2070,PAGO PAGO,...,2362.0,HAWAII DISTRICT OFFICE,"b $350,000-1 million",100 to 249,"POB 2070, PAGO PAGO, AS",-14.275632,-170.702036,NaN,600109506002001,600109506002001
3,19448,19448,19448,2530867306,4/29/20,951,PPP,NANA'S INC,POB 5350,PAGO PAGO,...,2361.0,HAWAII DISTRICT OFFICE,"b $350,000-1 million",100 to 249,"POB 5350, PAGO PAGO, AS",-14.275632,-170.702036,NaN,600109506002001,600109506002001
4,19449,19449,19449,2121297104,4/10/20,951,PPP,AMERICAN SAMOA 2000 INC,PO BOX 4047,PAGO PAGO,...,NaN,HAWAII DISTRICT OFFICE,"b $350,000-1 million",100 to 249,"PO BOX 4047, PAGO PAGO, AS",-14.275632,-170.702036,NaN,600109506002001,600109506002001
5,19450,19450,19450,4621778206,8/6/20,951,PPP,"PETER E REID STEVEDORING, INC.",1030 FAGATOGO,PAGO PAGO,...,4841.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",100 to 249,"1030 FAGATOGO, PAGO PAGO, AS",-14.279568,-170.692331,NaN,600109507002001,600109507002001
6,19451,19451,19451,4195837200,4/27/20,951,PPP,FORSGREN LTD. INC,POB 1358,PAGO PAGO,...,NaN,HAWAII DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"POB 1358, PAGO PAGO, AS",-14.275632,-170.702036,NaN,600109506002001,600109506002001
7,19452,19452,19452,4181737204,4/27/20,951,PPP,OTTOVILLE INVESTMENTS ONE INC,POB 999 OTTOVILLE ROAD,PAGOPAGO,...,7211.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"POB 999 OTTOVILLE ROAD, PAGOPAGO, AS",-14.275632,-170.702036,NaN,600109506002001,600109506002001
8,19453,19453,19453,1741728203,7/30/20,951,PPP,CONGREGATION CHRISTIAN CHURCH IN AMERICAN SAMOA,POB 1537,PAGO PAGO,...,8131.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"POB 1537, PAGO PAGO, AS",-14.275632,-170.702036,NaN,600109506002001,600109506002001
9,19454,19454,19454,2520817309,4/29/20,951,PPP,HARBOR MARITIME & STEVEDORING CO,POB 5350,PAGO PAGO,...,4883.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",100 to 249,"POB 5350, PAGO PAGO, AS",-14.275632,-170.702036,NaN,600109506002001,600109506002001


Drop Extra FIPS Columns

In [18]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
#fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,19445,19445,19445,6927458108,7/22/20,951,PPP,AST TELECOM LLC,478 Laufou Shopping Centre,PAGO PAGO,...,999 Other,5179 Other Telecommunications,5179.0,HAWAII DISTRICT OFFICE,"b $350,000-1 million",100 to 249,"478 Laufou Shopping Centre, PAGO PAGO, AS",-14.312881,-170.714627,600109509003002
1,19446,19446,19446,7124948005,6/30/20,951,PPP,MC CONNELL DOWELL,4669 TAFUNA,PAGO PAGO,...,999 Other,2362 Nonresidential Building Construction,2362.0,HAWAII DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"4669 TAFUNA, PAGO PAGO, AS",-14.331296,-170.726985,600509512021009
2,19447,19447,19447,1760228005,6/23/20,951,PPP,PARAMOUNT BUILDERS LIMITED,POB 2070,PAGO PAGO,...,999 Other,2362 Nonresidential Building Construction,2362.0,HAWAII DISTRICT OFFICE,"b $350,000-1 million",100 to 249,"POB 2070, PAGO PAGO, AS",-14.275632,-170.702036,600109506002001
3,19448,19448,19448,2530867306,4/29/20,951,PPP,NANA'S INC,POB 5350,PAGO PAGO,...,999 Other,2361 Residential Building Construction,2361.0,HAWAII DISTRICT OFFICE,"b $350,000-1 million",100 to 249,"POB 5350, PAGO PAGO, AS",-14.275632,-170.702036,600109506002001
4,19449,19449,19449,2121297104,4/10/20,951,PPP,AMERICAN SAMOA 2000 INC,PO BOX 4047,PAGO PAGO,...,NaN,NaN,NaN,HAWAII DISTRICT OFFICE,"b $350,000-1 million",100 to 249,"PO BOX 4047, PAGO PAGO, AS",-14.275632,-170.702036,600109506002001
5,19450,19450,19450,4621778206,8/6/20,951,PPP,"PETER E REID STEVEDORING, INC.",1030 FAGATOGO,PAGO PAGO,...,999 Other,4841 General Freight Trucking,4841.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",100 to 249,"1030 FAGATOGO, PAGO PAGO, AS",-14.279568,-170.692331,600109507002001
6,19451,19451,19451,4195837200,4/27/20,951,PPP,FORSGREN LTD. INC,POB 1358,PAGO PAGO,...,NaN,NaN,NaN,HAWAII DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"POB 1358, PAGO PAGO, AS",-14.275632,-170.702036,600109506002001
7,19452,19452,19452,4181737204,4/27/20,951,PPP,OTTOVILLE INVESTMENTS ONE INC,POB 999 OTTOVILLE ROAD,PAGOPAGO,...,999 Other,7211 Traveler Accommodation,7211.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"POB 999 OTTOVILLE ROAD, PAGOPAGO, AS",-14.275632,-170.702036,600109506002001
8,19453,19453,19453,1741728203,7/30/20,951,PPP,CONGREGATION CHRISTIAN CHURCH IN AMERICAN SAMOA,POB 1537,PAGO PAGO,...,999 Other,8131 Religious Organizations,8131.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",50 to 99,"POB 1537, PAGO PAGO, AS",-14.275632,-170.702036,600109506002001
9,19454,19454,19454,2520817309,4/29/20,951,PPP,HARBOR MARITIME & STEVEDORING CO,POB 5350,PAGO PAGO,...,999 Other,4883 Support Activities for Water Transportation,4883.0,HAWAII DISTRICT OFFICE,"a $150,000-350,000",100 to 249,"POB 5350, PAGO PAGO, AS",-14.275632,-170.702036,600109506002001


In [19]:
#fips_merge_drop = fips_merge.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop.loc[fips_merge["FIPS_z"].isnull()] #verify #27 , notna #22799 / 22826

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z


Convert to File

In [20]:
fips_AS = fips_merge_drop

In [21]:
fips_AS.to_csv("../../data/state_data/geo/geo_fips/AS_fips_scraped.csv") 

Review & Compare

In [22]:
geo_AS_fips = pd.read_csv("../../data/state_data/geo/geo_fips/AS_fips_scraped.csv") 
geo_AS_fips.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,19445,19445,19445,6927458108,7/22/20,951,PPP,AST TELECOM LLC,478 Laufou Shopping Centre,...,999 Other,5179 Other Telecommunications,5179.0,HAWAII DISTRICT OFFICE,"b $350,000-1 million",100 to 249,"478 Laufou Shopping Centre, PAGO PAGO, AS",-14.312881,-170.714627,600109509003002
1,1,19446,19446,19446,7124948005,6/30/20,951,PPP,MC CONNELL DOWELL,4669 TAFUNA,...,999 Other,2362 Nonresidential Building Construction,2362.0,HAWAII DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"4669 TAFUNA, PAGO PAGO, AS",-14.331296,-170.726985,600509512021009
2,2,19447,19447,19447,1760228005,6/23/20,951,PPP,PARAMOUNT BUILDERS LIMITED,POB 2070,...,999 Other,2362 Nonresidential Building Construction,2362.0,HAWAII DISTRICT OFFICE,"b $350,000-1 million",100 to 249,"POB 2070, PAGO PAGO, AS",-14.275632,-170.702036,600109506002001
3,3,19448,19448,19448,2530867306,4/29/20,951,PPP,NANA'S INC,POB 5350,...,999 Other,2361 Residential Building Construction,2361.0,HAWAII DISTRICT OFFICE,"b $350,000-1 million",100 to 249,"POB 5350, PAGO PAGO, AS",-14.275632,-170.702036,600109506002001
4,4,19449,19449,19449,2121297104,4/10/20,951,PPP,AMERICAN SAMOA 2000 INC,PO BOX 4047,...,NaN,NaN,NaN,HAWAII DISTRICT OFFICE,"b $350,000-1 million",100 to 249,"PO BOX 4047, PAGO PAGO, AS",-14.275632,-170.702036,600109506002001


In [23]:
##### fill in missing Lat, Long, FIPS
geo_AS_fips.loc[geo_AS_fips["FIPS_z"].isnull()] #27

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z


In [24]:
geo_AS_fips["BusinessType"] = geo_AS_fips["BusinessType"].fillna('Not Available')
geo_AS_fips["BusinessType"].unique()
geo_AS_fips.to_csv("../../data/state_data/geo/geo_fips/AS_fips_scraped.csv") 
geo_AS_fips =pd.read_csv("../../data/state_data/geo/geo_fips/AS_fips_scraped.csv") 
geo_AS_fips["BusinessType"].unique()

array(['Limited Liability Company(LLC)', 'Corporation',
       'Non-Profit Organization'], dtype=object)